In [1]:
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_functions_agent

from typing import Annotated
from prettytable import PrettyTable

PROCESS_MEMORY_AGENT_PROMPT = """
You are a memory expert. You can store and retrieve memories. You look for important information from the given user context and store it in your memory for future reference. You can also retrieve the stored memory when needed. 

You use the `store` tool with the key and memory as the arguments to store the memory. Moreover you use `ask_user_approval` tool to ask for user approval to update memory.

For example if user says: 'I am John Doe (23/M), I am a software engineer at XYZ company', you will store:
`key: user, memory: John Doe (23/M)`, `key: profession, memory: software engineer`, `key: company, memory: XYZ company`.

You have two options:
1. You add a new (key, information) pair to the memory if the key does not exist in the memory.
2. In very rare cases, you update the alredy existing information for a key if you find an updated value. But you "must" ask user for confirmation before overwriting the information using the `ask_user_approval` tool, if denied, add that information to a new key.

You try to keep the memory very concise storing only the important information.

You store personal inromation related to the user like `name`, `username`, `password`, `age`, `dob`, `gender`, `email`, etc.

Current memory: {memory}

User context: {context}

Remeber:
- You store the important infromation only.
- You mostly add new key, info pair
- Very rarely update the existing information, but ask user for approval before updating.
"""

In [2]:
process_memory = {}

In [3]:
@tool
def store(
    key: Annotated[str, "Key to store"],
    information: Annotated[str, "Memory to store"]
) -> None:
    """Store the memory with the given key."""
    process_memory[key] = information

@tool
def ask_user_approval(
    query: Annotated[str, "The query for which user approval is required."]
) -> Annotated[bool, "The user approval for the query."]:
    """Asks user approval for the given query."""
    print('Asking user approval for the query: ', query)
    response = input("Do you approve this action? (y/n): ")
    return response.lower() in ['y', 'yes']

In [4]:
class ProcessMemoryAgent:
    def __init__(self, llm:ChatOpenAI) -> None:
        self.llm = llm
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", PROCESS_MEMORY_AGENT_PROMPT),
                MessagesPlaceholder(variable_name="agent_scratchpad"),
            ]
        )

        
        tool = [store, ask_user_approval]
        agent = create_openai_functions_agent(self.llm, tool, prompt)
        self.executor = AgentExecutor(
            agent=agent, tools=tool, handle_parsing_errors=True, verbose=False
        )

    def retrieve_memory(self):
        table = PrettyTable()
        table.field_names = ["Key", "Information"]
        for key, information in process_memory.items():
            table.add_row([key, information])
        memory = table.get_string()
        # print("\033[33m" + memory + "\033[0m")
        return memory
    
    def get_keys(self):
        return process_memory.keys()

    def add_to_memory(self, context:str):
        self.executor.invoke({
            "context": context,
            "memory": self.retrieve_memory()
        })

In [5]:

llm = ChatOpenAI(model="gpt-4o", temperature=0)
process_memory_agent = ProcessMemoryAgent(llm)

In [6]:
process_memory_agent.add_to_memory("I am Sahasra Ranjan (23/M), a software engineer at XYZ company. I spend my days immersed in complex algorithms, creating innovative solutions that drive our digital world forward. In the midst of this technological whirlwind, I find solace in the simple pleasures of life. I am an avid guitarist, the melodies I create are a reflection of my inner world, a symphony of thoughts and emotions. Reading is another passion of mine, it's like a portal to different worlds, each book a new adventure. But enough about me, let's talk about the universe. Did you know that there are more stars in the universe than grains of sand on all the beaches on Earth? Or that a teaspoon of a neutron star would weigh about 10 million tons? Fascinating, isn't it? And what about the fact that we share 50% of our DNA with bananas? Makes you think, doesn't it?")


In [7]:
print(process_memory_agent.retrieve_memory())

+------------+-----------------------+
|    Key     |      Information      |
+------------+-----------------------+
|    user    | Sahasra Ranjan (23/M) |
| profession |   software engineer   |
|  company   |      XYZ company      |
|   hobby1   |     avid guitarist    |
|   hobby2   |        reading        |
+------------+-----------------------+


In [8]:
process_memory_agent.add_to_memory("I am John Doe")

Asking user approval for the query:  It seems your name has changed from Sahasra Ranjan to John Doe. Do you want to update this information?


In [9]:
print(process_memory_agent.retrieve_memory())

+------------+-------------------+
|    Key     |    Information    |
+------------+-------------------+
|    user    |      John Doe     |
| profession | software engineer |
|  company   |    XYZ company    |
|   hobby1   |   avid guitarist  |
|   hobby2   |      reading      |
+------------+-------------------+
